# TODO

- vector_store.ipynb
    - data/소설들.pdf 들 Vector store에 저장.
        - chroma, pinecone (알아서)
        - metadata 
            - title: 소설제목
            - author: 저자
            - full_text: 소설 전체 내용
        - Document(page_content="split된 내용", metadata={})
        - page_content는 조회할 때 사용할 embedding vector를 생성할 때 사용될 내용.
        - llm 모델에 전송할 context는 metadata의 full_text

- app.py
    - streamlit을 이용해서 서비스하는 application
    - vector_store.ipynb에서 구축한 DB를 이용해 질문과 관련된 내용들을 
      조회해서 llm에 전송하고 그 결과를 채팅창에 출력.
    - memory 기능은 사용하지 않는다.

# 소설들을 Vector Store에 저장

## Load
- `data/*.pdf` 파일들을 모두 load

- 각 소설들을 split 한 뒤 vector store에 저장한다.   
- metadata:
    - title: 소설제목
    - author: 저자
    - full_text: 소설 전체 내용

In [1]:
from glob import glob
glob("data/*.pdf")

['data\\금_따는_콩밭_김유정.pdf',
 'data\\동백꽃_김유정.pdf',
 'data\\메밀꽃_필_무렵_이효석.pdf',
 'data\\배따라기_김동인.pdf',
 'data\\백치_아다다_계용묵.pdf',
 'data\\벙어리_삼룡이_나도향.pdf',
 'data\\봄봄_김유정.pdf',
 'data\\소나기.pdf',
 'data\\술_권하는_사회_현진건.pdf',
 'data\\운수_좋은_날_현진건.pdf']

In [2]:
import re
import os
import config
from glob import glob

from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from dotenv import load_dotenv

load_dotenv()

# 설정 정보 읽기
CHUNK_SIZE = config.chunk_size
CHUNK_OVERLAP = config.chunk_overlap

MODEL_NAME  = config.model_name
EMBEDDING_NAME = config.embedding_name

COLLECTION_NAME = config.collection_name
PERSIST_DIRECTORY = config.persist_directory

# 소설 파일들 경로 조회
path_list = glob("data/*.pdf")

document_list = []

# load -> 전처리 -> split -> metadata 추가 
for path in path_list:
    # Document Load
    loader = PyMuPDFLoader(path)
    load_docs = loader.load()  # list[Document]

    # 전처리 - 페이지 번호 제거, 🙝🙟 를 \n\n 으로 변경
    full_text = [doc.page_content for doc in load_docs] # page_content를 합치기(1).
    full_text = ''.join(full_text)      # page_content를 합치기(2).
    full_text = full_text.replace("🙝🙟", "\n\n")
    full_text = re.sub(r"\d+\n", " ", full_text)   # 페이지 번호 제거
    
    # Split
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        model_name=MODEL_NAME,
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
    )
    # splitter.split_document([Document, Document,...]) -> list[Document]
    docs = splitter.split_text(full_text)

    # Metadata 생성
    author = os.path.splitext(os.path.basename(path))[0].split('_')[-1]
    metadata = {
        "title":load_docs[0].metadata["title"],
        "author": author,
        "full_text":full_text
    }

    # Document 생성
    for doc in docs:  # docs : split 된 소절 내용들. list[str]
        _doc = Document(metadata=metadata, page_content=doc)
        document_list.append(_doc)

print(len(document_list))

107


## Vector Store 저장
- Chroma Vector Store에 저장

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma


embedding_model = OpenAIEmbeddings(
    model=EMBEDDING_NAME
)

# [생성]/연결 + document들 추가.
vector_store = Chroma.from_documents(
    documents=document_list,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

# 확인

In [4]:
# 자정 개수 확인 (chroma 에서만 지원함.)
vector_store._collection.count()

107

In [5]:
###########################
# 검색

query = "현진건의 소설들의 특징은 어떤 것이 있을까?" 
query = "운수좋은 날에서 주인공은 왜 괴상하게도 운수가 좋다고 느꼈을까?"
query = "허 생원은 어느 장으로 가서 장사를 할 예정인가?"
query = "이효석의 메밀꽃 필 무렵의 주인공은 누구인가?"
query = "벙어리 삼룡이는 화자가 몇살 때 일인가?"
query = "삼룡이의 주인 아들의 색시는 몇살인가?"

result = vector_store.similarity_search(
    query,
    k=5,
    # filter={"author": "현진건"}
)

In [6]:
for res in result:
    print(res.metadata['title'], res.metadata['author'], sep=" | ")
    print(res.page_content)
    print("=====================================")

벙어리 삼룡이 | 나도향
이리하여 색시는 시집오던 날부터 팔자 한탄을 하고서 날마
다 밤마다 우는 사람이 되었다.
울며는 요사스럽다고 때린다. 또 말이 없으면, 빙충맞다고
친다. 이리하여 그 집에는 평화스러운 날이 하루도 없었다.
이것을 날마다 보는 사람 가운데 알 수 없는 의혹을 품게 된
사람이 하나 있으니 그는 곧 벙어리 삼룡이었다.
그렇게 예쁘고 유순하고 그렇게 얌전한 벙어리의 눈으로 보
아서는 감히 손도 대지 못할 만치 선녀 같은 색시를 때리는
 것은 자기의 생각으로는 도저히 풀 수 없는 의심이다.
보기에는 황홀하고 건드리기도 황홀할 만치 숭고한 여자를
그렇게 하대한다는 것은 너무나 세상에 있지 못할 일이다.
자기는 주인 새서방에게 개나 돼지같이 얻어맞는 것이 마땅
한 이상으로 마땅하지마는 선녀와 짐승의 차가 있는 색시와
자기가 똑같이 얻어맞는 것은 너무 무서운 일이다. 어린 주
인이 천벌이나 받지 않을까 두렵기까지 하였다.
어떠한 달밤, 사면은 고요적막하고 별들은 드문드문 눈들만
깜박이며 반달이 공중에 뚜렷이 달려 있어 수은으로 세상을
깨끗하게 닦아낸 듯이 청명한데 삼룡이는 검둥개 등을 쓰다
듬으며 밖 마당 멍석 위에 비슷이 드러누워 하늘을 쳐다보며
생각하여 보았다.
주인 색시를 생각하면 공중에 있는 달보다도 더 곱고 별들보
다도 더 깨끗하였다. 주인 색시를 생각하면 달이 보이고 별
이 보이었다. 삼라만상을 씻어내는 은빛보다도 더 흰 달이나
별의 광채보다도 그의 마음이 아름답고 부드러운 듯하였다.
마치 달이나 별이 땅에 떨어져 주인 새아씨가 된 것도 같고
주인 새아씨가 하늘에 올라가면 달이 되고 별이 될 것 같았
다.
벙어리 삼룡이 | 나도향
그해 가을이다. 주인의 아들이 장가를 들었다. 색시는 신랑
보다 두 살 위인 열 아홉 살이다. 주인이 본시 자기가 언제든
지 문벌이 얕은 것을 한탄하여 신부를 구할 때에 첫째 조건
이 문벌이 높아야 할 것이었다. 그러나 문벌 있는 집에서는
그리 쉽게 색시를 내놀 리가 없었다.
그러므로 하는 수없이 그 어떠한 영락

In [ ]:
from textwrap import dedent # 들여쓰기 복원 함수.
